# <font face="Times New Roman" size="6">Transferencia de Calor y Masa</font>
# <font face="Times New Roman" size="6" color='gray'>Conducto de calor (heat pipe)</font>   

## Enunciado

Se desea evaluar la habilidad de un caloducto para transferir energía térmica.
El mismo consiste en un tubo de cobre cerrado por ambos extremos, el cual tiene una longitud de 1.5 m y un diámetro de 2.5 cm. El tubo contiene tolueno líquido.
El tramo inferior de 5 cm de tubo se enrolla con una cinta calefactora la cual provee 100 W de calor al líquido. El tolueno entonces se evapora en el extremo inferior del tubo y el vapor se desplaza al extremo superior donde condensa al entrar en contacto con la superficie fría del tubo.
El tramo superior de 6 cm de tubo se mantiene a 29°C por medio de un flujo de agua a 25°C y 1 atm. El condensado de tolueno vuelve a fluir hacia el tramo inferior del tubo y el ciclo se repite.
El caloducto se encuentra aislado térmicamente excepto en su tramo inferior en contacto con el calentador y en su tramo superior en contacto con el agua.

**(a)** Estime la temperatura de saturación y la presión del tolueno en el caloducto.<br>
**(b)** Estime la temperatura de la superficie del tubo que se encuentra en contacto con el calentador.<br>
**(c)** Estime la velocidad del agua de enfriamiento.<br>
**(d)** Determine la conductividad térmica efectiva del caloducto.

In [14]:
import numpy as np
import IPython.display as ipd
from matplotlib import pyplot as plt
from scipy import optimize
import ht
import CoolProp as cp


# *********** Datos ***********
# Aceleracion de la gravedad
g = 9.8 # [m/s^2]
# Geometria del caloducto

L = 1.5 # longitud del tubo [m]
D = 25e-3 # diametro externo del tubo [m]
L_h = 50e-3 # longitud de la seccion caliente [m]
A_h = np.pi*D*L_h # area superficial de la seccion caliente [m^2]
L_c = 60e-3 # longitud de la seccion fria [m]
A_c = np.pi*D*L_c # area superficial de la seccion fria [m^2]

# Condiciones de operacion

T_inf=25.0+273 # temperatura del agua [K]
q_dot=100.0 # velocidad de transferencia de calor provista por el calentador [W]
T_ts=29.0+273 # temperatura de la superficie del tubo en su tramo superior [K]

## Proceso de Condensación

La condensación en película se produce en la sección ***fría*** del caloducto.
A fin de obtener el coeficiente de transferencia de calor para la condensación en película sobre el interior de un tubo horizontal ($\bar{h}_{c}$), recurrimos a la correlación de Chato, 1962 (Ver Seccion 10.10 de <a name="ref-1"/>[Incropera, 2007](#cite-incropera2007fundamentals)):

\begin{equation}
\bar{h}_{c}=0.555\left[\frac{g \rho_{l,sat}\left(\rho_{l,sat}-\rho_{v,sat}\right) k_{l,sat}^{3} \Delta i_{vap}^{\prime}}{\mu_{l,sat}\left(T_{sat}-T_{ts}\right) D}\right]^{1/4}\;\;,
\;\;\;\;\;\;Re_{v,i}<35\cdot10^{3}
\end{equation}

donde $Re_{v,i}$ refiere al número de Reynolds para el flujo de vapor evaluado en la sección de entrada del tubo, mientras que $\Delta i_{vap}^{\prime}$ corresponde al calor latente de vaporización modificado dado por:

\begin{equation}
\Delta i_{vap}^{\prime}\equiv\Delta i_{vap}+\frac{3}{8}c_{l,sat}\left(T_{sat}-T_{ts}\right)
\end{equation}

Existe un balance entre la velocidad a la cual se transfiere calor a la sección de condensación ($\dot{q}_{c}$) y la velocidad de transferencia provista por el calentador ($\dot{q}_{h}$):

\begin{equation}
\dot{q}_{c}=\dot{q}_{h}=\dot{q}=A_{c}\;\bar{h}_{c}\left(T_{sat}-T_{ts}\right)
\end{equation}

### Procedimiento de cálculo

La temperatura de saturación ($T_{sat}$) no se especifica por lo cual debe calcularse. Esto conduce a un proceso iterativo a fin de evaluar las propiedades del tolueno y obtener el coeficiente de transferencia.
Como punto de partida, suponemos una $T_{sat}$ y evaluamos las propiedades del tolueno a esa temperatura recurriendo a la librería CoolProp. Calculamos entonces el coeficiente $\bar{h}_{c}$ y ajustamos el valor de $T_{sat}$ de forma tal de satisfacer el balance previo dado por la ley de enfriamiento de Newton.

Propiedades del Tolueno desde http://www.coolprop.org/v4/Fluids/Toluene.html
Las aplicamos a través de la librería CoolProp (cp)

In [15]:
def evaluar_prop(T_sat):
    
    # Presion de vapor del tolueno a T_sat
    
    P_sat = cp.CoolProp.PropsSI('P','T',T_sat, 'Q',1,'Toluene')


    # Densidad del vapor saturado
    
    rho_vsat = cp.CoolProp.PropsSI('D','T',T_sat, 'Q',1,'Toluene') # [kg/m^3]

    # Densidad del liquido saturado
    
    rho_lsat = cp.CoolProp.PropsSI('D','T',T_sat, 'Q',0,'Toluene') # [kg/m^3]
    

    # Calor latente de vaporizacion    
    
    
    i_v = cp.CoolProp.PropsSI('H','T',T_sat, 'Q',1,'Toluene') #
    i_l = cp.CoolProp.PropsSI('H','T',T_sat, 'Q',0,'Toluene') #
    Deltai_vap = (i_v - i_l) # [J/kg]
    

    # Viscosidad del liquido saturado
    
    mu_lsat = cp.CoolProp.PropsSI('V','T',T_sat, 'Q',0,'Toluene') # [Pa*s]
    

    # Conductividad termica del liquido saturado 
    
    k_lsat = cp.CoolProp.PropsSI('L','T',T_sat, 'Q',0,'Toluene') # [W/m*K]
    
    # calor especifico del liquido saturado
    
    c_lsat = cp.CoolProp.PropsSI('C','T',T_sat, 'Q',0,'Toluene') # [J/kg*K]

    return [P_sat,rho_vsat,rho_lsat,Deltai_vap,mu_lsat,k_lsat,c_lsat]


def corr_Chato(T_sat,rho_vsat,rho_lsat,Deltai_vap_m,mu_lsat,k_lsat,c_lsat,D):
    
    h_c = 0.555*((g*rho_lsat*(rho_lsat-rho_vsat)*(k_lsat**3)*Deltai_vap_m)/
                 (mu_lsat*(T_sat-T_ts)*D))**(1/4)
    
    return h_c


def condensado(T_sat):
    g =9.8
    # Evalua propiedades a T_sat 
    P_sat,rho_vsat,rho_lsat,Deltai_vap,mu_lsat,k_lsat,c_lsat = evaluar_prop(T_sat)

    
    # Calor latente de vaporizacion modificado [J/kg]
    Deltai_vap_m = Deltai_vap+(3.0/8.0)*c_lsat*(T_sat-T_ts)    

    # Coeficiente de transferencia [W/m^2*K]
    h_c = corr_Chato(T_sat,rho_vsat,rho_lsat,Deltai_vap_m,mu_lsat,k_lsat,c_lsat,D)
    
    delta_condensado = k_lsat / h_c    
    
    masa_condensado = rho_lsat*(rho_lsat-rho_vsat)/(3*mu_lsat)*g*delta_condensado**3
    
    h_c2 = ht.Akers_Deans_Crosser(masa_condensado, rho_vsat, rho_lsat, k_lsat, mu_lsat, c_lsat, D, 1)
    # Ley de enfriamiento (func=0)
    func = h_c2 - q_dot/(A_c*(T_sat-T_ts))
    
    return func


# ******** Resultados ********

# Encuentra raiz de la funcion 'condensado' en un intervalo dado usando metodo de Brent
T_sat = optimize.brentq(condensado,T_ts+0.1,T_ts+100)

# Propiedades
P_sat,rho_vsat,rho_lsat,Deltai_vap,mu_lsat,k_lsat,c_lsat = evaluar_prop(T_sat)

# Calor latente modificado 
Deltai_vap_m = Deltai_vap+(3.0/8.0)*c_lsat*(T_sat-T_ts)
    
# Coef. de transferencia
h_c = corr_Chato(T_sat,rho_vsat,rho_lsat,Deltai_vap_m,mu_lsat,k_lsat,c_lsat,D)


#h_c = ht.condensation.Akers_Deans_Crosser(m, rhog, rhol, kl, mul, Cpl, D, x)

# Imprime
objeto1 = ipd.Math(r'\longrightarrow \bar{h}_{c} = %s'%format(h_c,'6.1f')+'\;W/m^{2}K')
objeto2 = ipd.Math(r'\longrightarrow T_{sat} = %s'%format(T_sat-273,'6.1f')+'\;°C')
objeto3 = ipd.Math(r'\longrightarrow P_{sat} = %s'%format(P_sat,'6.2f')+'\;Pa')
print('P_{sat} = %5.2e Pa'%P_sat )
ipd.display(objeto1,objeto2,objeto3)

P_{sat} = 9.66e+03 Pa


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Proceso de Evaporación

A fin de obtener el coeficiente de transferencia de calor para un evaporador que opera en el régimen de ebullición nucleada ($\bar{h}_{b}$), recurrimos a la correlación de Rohsenow, 1952 (Ver Sección 7.2.3 de <a name="ref-2"/>[Nellis, 2008](#cite-nellis_klein_2008)):

\begin{equation}
\bar{h}_{b}=\mu_{l,sat} h_{fg} \sqrt{\frac{g\left(\rho_{l,sat}-\rho_{v,sat}\right)}{\sigma}}\left[\frac{c_{l,sat} \left(T_{bs}-T_{sat}\right)^{2/3}}{C_{n b} h_{fg} P r_{l,sat}^{n}}\right]^{3}
\end{equation}



Reconozcamos algunas cantidades conocidas:

\begin{equation}
    \text{Ja} = \frac{c_{l,sat} (T_{bs}-T_{sat})}{h_{fg}} 
\end{equation}

\begin{equation}
\ell =  \sqrt{\frac{g\left(\rho_{l,sat}-\rho_{v,sat}\right)}{\sigma}}
\end{equation}



La velocidad de transferencia de calor está dada por la ley de enfriamiento de Newton:

\begin{equation}
\dot{q}=A_{h}\;\bar{h}_{b}\left(T_{bs}-T_{sat}\right)
\end{equation}

### Procedimiento de cálculo

La temperatura de la superficie del tubo en su tramo inferior ($T_{bs}$) no se especifica por lo cual debe calcularse. Esto conduce a un proceso iterativo a fin de obtener el coeficiente de transferencia.
Como punto de partida, suponemos una temperatura de pared $T_{bs}$ y calculamos
el coeficiente $\bar{h}_{b}$ recurriendo a la función `boiling_nucleic.Rohsenow` disponible en la librería Python [`ht`](https://pypi.org/project/ht/) la cual permite implementar la expresión previa. Finalmente ajustamos el valor de $T_{bs}$ de forma tal de satisfacer la ley de enfriamiento de Newton.

In [4]:
# Tension superficial 
sigma = cp.CoolProp.PropsSI('surface_tension','T',T_sat, 'Q',0,'Toluene') # [N/m]

# Coeficiente que depende de la combinacion superficie-fluido
Cnb = 0.013

# Exponente del numero de Prandtl
n = 1.7


In [5]:
def evaporado(T_bs):
    
    # Coeficiente de transferencia [W/m^2*K]
    h_b = ht.boiling_nucleic.Rohsenow(rhol=rho_lsat, rhog=rho_vsat, mul=mu_lsat, 
                                      kl=k_lsat, Cpl=c_lsat, Hvap=Deltai_vap,
                                      sigma=sigma, Te=T_bs-T_sat, Csf=Cnb, n=n)
    
    # Ley de enfriamiento (func=0)
    func = h_b - q_dot/(A_h*(T_bs-T_sat))
    return func
    

In [16]:
# ******** Resultados ********

# Encuentra raiz de la funcion 'evaporado' en un intervalo dado usando metodo de Brent
T_bs = optimize.brentq(evaporado,T_sat+0.1,T_sat+100)

# Coef. de transferencia
h_b = ht.boiling_nucleic.Rohsenow(rhol=rho_lsat, rhog=rho_vsat, mul=mu_lsat,
                                  kl=k_lsat, Cpl=c_lsat, Hvap=Deltai_vap,
                                  sigma=sigma, Te=T_bs-T_sat, Csf=Cnb, n=n)

# Imprime
objeto1 = ipd.Math(r'\longrightarrow T_{bs} = %s'%format(T_bs-273,'6.1f')+'\;°C')
objeto2 = ipd.Math(r'\longrightarrow \bar{h}_{b} = %s'%format(h_b,'6.1f')+'\;W/m^{2}K')
ipd.display(objeto1,objeto2)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Mecanismo de Convección

El coeficiente de transferencia para la convección externa al tubo se calcula por medio de:

\begin{equation}
\dot{q}=A_{c}\;\bar{h}_{conv}\left(T_{ts}-T_{\infty}\right)
\end{equation}

resultando $\bar{h}_{conv}=5305\;W/m^{2}K$. La velocidad requerida de agua de enfriamiento ($u_{\infty}$) se obtiene considerando la correlación de Churchill y Bernstein, 1977 (Ver Seccion 4.9.3 de <a name="ref-2"/>[Nellis, 2008](#cite-nellis_klein_2008)):

\begin{equation}
\overline{N u}_{D}=0.3+\frac{0.62 \operatorname{Re}_{D}^{0.5} \operatorname{Pr}^{1 / 3}}{\left[1+\left(\frac{0.4}{P r}\right)^{2 / 3}\right]^{0.25}}\left[1+\left(\frac{R e_{D}}{2.82 \times 10^{5}}\right)^{0.625}\right]^{0.80}
\end{equation}

la cual permite calcular el número de Nusselt ($\overline{Nu}_{D}$) para el flujo a través de un cilindro dados los números adimensionales $Re_{D}$ y $Pr$ evaluados a la temperatura de película $\left(T_{ts}+T_{\infty}\right)/2$.

### Procedimiento de cálculo

Como punto de partida del esquema iterativo, suponemos un número de Reynolds $Re_{D}$ y calculamos el número de Nusselt ($\overline{Nu}_{D}$) recurriendo a la función `conv_external.Nu_cylinder_Churchill_Bernstein` disponible en la librería Python [`ht`](https://pypi.org/project/ht/) la cual permite implementar la expresión previa. Finalmente ajustamos el valor de $Re_{D}$ de forma tal de satisfacer el valor de $\bar{h}_{conv}$ obtenido previamente al aplicar la ley de enfriamiento de Newton.

In [17]:
h_conv = q_dot/A_c/(T_ts-T_inf)
print('h_conveccion = %.2f'%h_conv)
T_ref = (T_ts+T_inf)/2

mu_agua = cp.CoolProp.PropsSI('V','T',T_ref,'Q',0,'water')
rho_agua = cp.CoolProp.PropsSI('D','T',T_ref,'Q',0,'water')
nu_agua = mu_agua/rho_agua
k_agua = cp.CoolProp.PropsSI('L','T',T_ref,'Q',0,'water')
cp_agua = cp.CoolProp.PropsSI('C','T',T_ref,'Q',0,'water')
a_agua = k_agua / (cp_agua*rho_agua)

Pr_agua = nu_agua / a_agua

h_conveccion = 5305.16


In [8]:
def solucion_conveccion (U):
    ReD = U*D*rho_agua / mu_agua
    
    NuD = ht.conv_external.Nu_cylinder_Churchill_Bernstein (ReD,Pr_agua)
    
    h_conv_aux = k_agua/D * NuD
    
    func = h_conv_aux - h_conv
    
    return func

In [18]:
U_agua = optimize.brentq(solucion_conveccion,0.0001,100)

print('Velocidad agua = %.2f'%U_agua)

Velocidad agua = 0.99


## Desempeño térmico del caloducto

La conductividad efectiva en este caso se refiere a la conductividad que debería tener una barra sólida de cobre con el mismo $\Delta T$ impuesto en ambos extremos para transferir energía a la misma velocidad que el caloducto, es decir:  

\begin{equation}
\dot{q}=\pi \frac{D^{2}}{4} \frac{k_{ef}}{L}\left(T_{bs}-T_{ts}\right)\end{equation}



In [19]:
k_ef = q_dot *4*L / (D**2*np.pi*(T_bs-T_ts))
print('Conductividad_efectiva k_ef =%.1f'%k_ef)

Conductividad_efectiva k_ef =5283.9


In [20]:
ht.insulation.k_material('Copper')

380.0

In [11]:
k_cobre = ht.insulation.k_material('Copper')
print('Conductividad_relativa al cobre = %.1f'%(k_ef/k_cobre))

Conductividad_relativa al cobre = 13.9


<div class="alert alert-block alert-warning">
<h2>Respuesta (d)</h2>       

Teniendo en cuenta que $k_{cobre}=396\;W/mK$ evaluada a una temperatura media $\left(T_{bs}+T_{ts}\right)/2$, podemos constatar la alta eficacia del caloducto para transferir energía térmica.
</div>

<!--bibtex

@book{incropera2007fundamentals,
  title={Fundamentals of heat and mass transfer},
  author={Incropera, Frank P and Lavine, Adrienne S and Bergman, Theodore L and DeWitt, David P},
  year={2007},
  publisher={Wiley}
}

@book{nellis_klein_2008, place={Cambridge}, title={Heat Transfer}, DOI={10.1017/CBO9780511841606}, publisher={Cambridge University Press}, author={Nellis, Gregory and Klein, Sanford}, year={2008}}

... Other Bibtex entries go here.

-->

<!--bibtex

@Article{PER-GRA:2007,
  Author    = {P\'erez, Fernando and Granger, Brian E.},
  Title     = {{IP}ython: a System for Interactive Scientific Computing},
  Journal   = {Computing in Science and Engineering},
  Volume    = {9},
  Number    = {3},
  Pages     = {21--29},
  month     = may,
  year      = 2007,
  url       = "http://ipython.org",
  ISSN      = "1521-9615",
  doi       = {10.1109/MCSE.2007.53},
  publisher = {IEEE Computer Society},
}

... Other Bibtex entries go here.

-->

# References

<a name="cite-incropera2007fundamentals"/><sup>[^](#ref-1) </sup>Incropera, Frank P and Lavine, Adrienne S and Bergman, Theodore L and DeWitt, David P. 2007. _Fundamentals of heat and mass transfer_.

<a name="cite-nellis_klein_2008"/><sup>[^](#ref-2) </sup>Nellis, Gregory and Klein, Sanford. 2008. _Heat Transfer_.



In [12]:
#Para conocer los alias de los fluidos que tiene incorporado CoolProp
cp.CoolProp.get_global_param_string("FluidsList").split(',')

['1-Butene',
 'Acetone',
 'Air',
 'Ammonia',
 'Argon',
 'Benzene',
 'CarbonDioxide',
 'CarbonMonoxide',
 'CarbonylSulfide',
 'CycloHexane',
 'CycloPropane',
 'Cyclopentane',
 'D4',
 'D5',
 'D6',
 'Deuterium',
 'Dichloroethane',
 'DiethylEther',
 'DimethylCarbonate',
 'DimethylEther',
 'Ethane',
 'Ethanol',
 'EthylBenzene',
 'Ethylene',
 'EthyleneOxide',
 'Fluorine',
 'HFE143m',
 'HeavyWater',
 'Helium',
 'Hydrogen',
 'HydrogenChloride',
 'HydrogenSulfide',
 'IsoButane',
 'IsoButene',
 'Isohexane',
 'Isopentane',
 'Krypton',
 'MD2M',
 'MD3M',
 'MD4M',
 'MDM',
 'MM',
 'Methane',
 'Methanol',
 'MethylLinoleate',
 'MethylLinolenate',
 'MethylOleate',
 'MethylPalmitate',
 'MethylStearate',
 'Neon',
 'Neopentane',
 'Nitrogen',
 'NitrousOxide',
 'Novec649',
 'OrthoDeuterium',
 'OrthoHydrogen',
 'Oxygen',
 'ParaDeuterium',
 'ParaHydrogen',
 'Propylene',
 'Propyne',
 'R11',
 'R113',
 'R114',
 'R115',
 'R116',
 'R12',
 'R123',
 'R1233zd(E)',
 'R1234yf',
 'R1234ze(E)',
 'R1234ze(Z)',
 'R124',
 'R

Para conocer las entradas  http://www.coolprop.org/coolprop/HighLevelAPI.html#parameter-table

In [21]:
ht.materials_dict.keys() ##lista de materiales


dict_keys(['Silica', 'Silica special', 'Fused silica', 'Fireclay', 'High-duty fireclay', 'Sillimanite', 'Mullite', 'Corundum 90%', 'Bauxite', 'Corundum 99%', 'Corundum Spinel', 'ACr 90', 'ACrZ 20', 'ACrZ 60', 'Magnesite Chrome', 'Magnesia', 'Magnesite Spinel', 'Magnesite Graphite H15', 'Dolomite P10', 'Sillimanite P5', 'Bauxite P5', 'Corundum P10', 'Magnesite P5', 'Zirconia', 'Zircon', 'AZS 41', 'AZS 33', 'a/b-Alumina', 'SIC 40%', 'SIC 70%', 'SIC 90%', 'L1260', 'L1400', 'L1540', 'L1760', 'L1870', 'Carbon, anthracite', 'Carbon, graphite', 'Board, Asbestos/cement', 'Board, Cement', 'Board, Fiber/cement, 1400 kg/m^3', 'Board, Fiber/cement, 1000 kg/m^3', 'Board, Fiber/cement, 400 kg/m^3', 'Board, Fiber/cement, 300 kg/m^3', 'Gypsum or plaster board', 'Oriented strand board (OSB)', 'Plywood (douglas fir)', 'Plywood/wood panels', 'Vegetable fiber board, Sheathing, regular density', 'Vegetable fiber board, Sheathing, intermediate density', 'Vegetable fiber board, Nail-base sheathing', 'Vegetab